In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow import keras

In [ ]:
# Function to load images and labels from text files
def load_data(data_dir, img_width, img_height):
    images = []
    labels = []
    label_mapping = {'female': 0, 'male': 1}
    for folder in os.listdir(data_dir):
        if folder.startswith('.'):  # Skip hidden folders
            continue
        folder_path = os.path.join(data_dir, folder)
        label = label_mapping[folder.lower()]  # Get the integer label
        for file in os.listdir(folder_path):
            if file.endswith('.jpg'):
                image_path = os.path.join(folder_path, file)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (img_width, img_height))  # Resize images
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
# Load and preprocess the dataset
path = '/content/drive/MyDrive/Colab Notebooks/gender.v1i.folder/train'
images, labels = load_data(path,128,128)
labels = labels.astype(np.int32)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Assuming your model is called 'my_model'
model.save('my_model.h5')

In [ ]:
# Function to preprocess user input image
def preprocess_image(image_path, target_size):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make prediction using the loaded model
def predict_gender(image_array, model):
    # Perform prediction
    prediction = model.predict(image_array)
    # Map prediction to class label
    if prediction[0] > 0.5:
        gender = 'Male'
    else:
        gender = 'Female'
    return gender

In [ ]:
# Load the trained model
model = load_model('trained_model.h5')

In [ ]:
# Preprocess user input image
image_path = ''
input_size = (128, 128)  # Adjust this size according to your model's input size
processed_image = preprocess_image(image_path, input_size)

# Perform prediction
predicted_gender = predict_gender(processed_image, model)
print("Predicted gender:", predicted_gender)